In [28]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
 
    
try:
    df = pd.read_csv('../debug_programados_del_dia.csv', encoding='utf-8',  sep='|', engine='python')
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")

   doc_id    doc_emp  doc_num_gr doc_num_fac             doc_fec_crea  \
0   84505  FIAMASTER   T002-1940         NaN  2025-08-20 08:26:47.714   
1   84468  FIAMASTER   T002-1941         NaN  2025-08-19 14:47:37.105   
2   84479  FIAMASTER   T002-1942         NaN  2025-08-19 15:25:25.441   
3   84487  FIAMASTER   T002-1943         NaN  2025-08-19 16:11:48.480   
4   84449  FIAMASTER  T007-36511  F007-68839  2025-08-19 11:36:01.755   

  doc_fec_ent_gr doc_fec_prog_gr           doc_raz_soc  \
0     2025-08-20      2025-08-20        FIAMASTER S.A.   
1     2025-08-20      2025-08-20        FIAMASTER S.A.   
2     2025-08-20      2025-08-20        FIAMASTER S.A.   
3     2025-08-20      2025-08-20        FIAMASTER S.A.   
4     2025-08-20      2025-08-15  COMERCIAL ANGULO SRL   

                        doc_con_pag  doc_ag_trans  ...          rd_hora_sal  \
0  NO USAR - Contado contra entrega           NaN  ...  2025-08-20 12:29:56   
1  NO USAR - Contado contra entrega           NaN  ...

In [ ]:
def save_dataframe_as_image(df, title, filename="reporte_despacho.png"):
    """Renderiza el DataFrame como una tabla en Matplotlib y la guarda como imagen."""
    
    # Crea una figura y un eje
    # Ajusta el tamaño de la figura en función del número de filas para evitar el recorte
    fig, ax = plt.subplots(figsize=(7, len(df) * 0.5)) 
    ax.axis('tight')
    ax.axis('off')

    # Convierte todos los datos a string para la representación de la tabla
    cell_data = df.values.astype(str)
    
    # Crea la tabla de matplotlib
    table = ax.table(cellText=cell_data,
                     colLabels=df.columns,
                     loc='center',
                     cellLoc='center')

    # Estilizado
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    border_color = "#C7D2DD" # Gris azulado suave
    header_bg_color = '#4682B4' # Azul fuerte para la cabecera
    total_row_bg_color = '#E0E0E0' # Gris un poco más oscuro para la fila total
    
    
    # Aplicar estilos a la cabecera
    for (row, col), cell in table.get_celld().items():
        cell.set_edgecolor(border_color) # Borde para todas las celdas
        
        if row == 0: # Cabecera
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor(header_bg_color) 
            # Borde más pronunciado para la cabecera si se desea
            cell.set_linewidth(1.5) 
        elif row == len(df): # Fila 'Total general'
            cell.set_text_props(weight='bold')
            cell.set_facecolor(total_row_bg_color) 
            cell.set_linewidth(1.5) # Borde más pronunciado para la fila total
        else: # Celdas de datos
            cell.set_facecolor('white') # Fondo blanco para las celdas de datos
            cell.set_linewidth(0.5) # Bordes más finos para las celdas de datos
            
    # Ajuste para los bordes externos de toda la tabla si se necesitan más gruesos
    # Esto puede hacerse configurando los bordes del propio objeto table, o
    # asegurando que las celdas en los bordes externos tengan un linewidth mayor.
    # Por ejemplo, para la caja exterior:
    for i in range(len(df)):
        for j in range(len(df.columns)):
            cell = table[(i, j)]
            if i == 0: # Borde superior de la tabla
                cell.set_linewidth(1.5)
            if i == len(df) : # Borde inferior de la tabla
                cell.set_linewidth(1.5)
            if j == 0: # Borde izquierdo de la tabla
                cell.set_linewidth(1.5)
            if j == len(df.columns) : # Borde derecho de la tabla
                cell.set_linewidth(1.5)


    # Agregar título
    plt.title(title, fontsize=14, fontweight='bold', pad=15)

    # Guardar la imagen (El entorno de ejecución la mostrará)
    plt.savefig(filename, bbox_inches='tight', dpi=150)
    plt.close(fig) 
    # Imprimir el DataFrame para verificación en la consola
    print("--- Contenido del DataFrame antes de guardar como imagen (Verificación) ---")
    print(df.to_string(index=False))
    print("\nReporte guardado como 'reporte_despacho.png' y mostrado a continuación.")
    

In [46]:

# Limpieza de datos (Asegurando que los NaN sean 0 para la suma)
import datetime


df['doc_peso_tot'] = pd.to_numeric(df['doc_peso_tot'], errors='coerce').fillna(0)
df['rut_id'] = pd.to_numeric(df['rut_id'], errors='coerce').fillna(0)

df_grouped = df.groupby('veh_placa').agg(
    Peso_Total=('doc_peso_tot', 'sum'),
    cantidad_viajes=('rut_id', 'nunique') 
).reset_index()

# 2.2. Renombrar la columna de vehículos (para coincidir con la imagen)
df_grouped = df_grouped.rename(columns={'veh_placa': 'Vehiculos'})

# 2.3. Calcular el Peso Total General
total_general_peso = df_grouped['Peso_Total'].sum()

# 2.4. Calcular el Porcentaje
df_grouped['Avance %'] = (df_grouped['Peso_Total'] / total_general_peso)

 
# 3.1. Ordenar por el nombre del vehículo
df_grouped = df_grouped.sort_values(by='Peso_Total').reset_index(drop=True)

# 3.2. Crear el Total General (fila)
# ************** CORRECCIÓN CLAVE: Usar df_grouped, no df **************
# Calcular el Total General de Viajes sumando la columna ya agrupada.
total_rutas = df_grouped['cantidad_viajes'].sum() 

total_row = pd.DataFrame([{
    'Vehiculos': 'Total general',
    'Peso_Total': total_general_peso,
    'Avance %': 1.0,
    # Usamos el mismo nombre de columna que en df_grouped para estandarizar
    'cantidad_viajes': total_rutas 
}])

# 3.3. Concatenar la fila de total
df_final = pd.concat([df_grouped, total_row], ignore_index=True)


def format_row(row):
    # Formatear el peso
    if row['Vehiculos'] == 'Total general':
        # Mostrar el total en toneladas (X.Xt)
        peso_str = f"{row['Peso_Total'] / 1000:.1f} t"
 
    elif row['Peso_Total'] >= 1000:
        # Mostrar pesos de 1000 a 1999 en toneladas
        peso_str = f"{row['Peso_Total'] / 1000:.2f} t"
    else:
        # Mostrar pesos pequeños en kg sin decimales
        peso_str = f"{row['Peso_Total']:.0f} kg"
        
    # Formatear el porcentaje
    porcentaje_str = f"{row['Avance %']:.2%}"
    
    # Formatear la Cantidad de viajes (ahora la columna es 'cantidad_viajes' en todos los casos)
    ruta_val = int(row['cantidad_viajes'])

    return pd.Series({
        'Vehiculos': row['Vehiculos'],
        'Peso Total': peso_str,
        'Avance %': porcentaje_str, # Nombre de columna de la imagen
        'Cant. de viajes': ruta_val # Usamos 'Cantidad de viajes' para coincidir con la imagen final
    })

# Aplicar el formato
# Renombramos las columnas del DF estilizado para usar los nombres de la imagen (%, Ruta)
df_styled = df_final.apply(format_row, axis=1)

# --- 4. Resultado (Impresión) ---
report_title = f"DESPACHOS"

print(df_styled.to_string(index=False))

save_dataframe_as_image(df_styled, report_title)

    Vehiculos Peso Total Avance %  Cant. de viajes
      BSQ-823     631 kg    5.49%                1
      CCA-828     1.65 t   14.33%                1
      BSR-797     1.99 t   17.32%                1
      BSS-818     2.20 t   19.11%                1
      CCB-949     2.20 t   19.11%                1
      BSS-795     2.83 t   24.64%                2
Total general     11.5 t  100.00%                7
--- Contenido del DataFrame antes de guardar como imagen (Verificación) ---
    Vehiculos Peso Total Avance %  Cant. de viajes
      BSQ-823     631 kg    5.49%                1
      CCA-828     1.65 t   14.33%                1
      BSR-797     1.99 t   17.32%                1
      BSS-818     2.20 t   19.11%                1
      CCB-949     2.20 t   19.11%                1
      BSS-795     2.83 t   24.64%                2
Total general     11.5 t  100.00%                7

Reporte guardado como 'reporte_despacho.png' y mostrado a continuación.
